# Configuration 

In [3]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = "gpt-4o-mini"  

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT


# Create client for Azure OpenAI
client = OpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    base_url=f"{os.environ['AZURE_OPENAI_ENDPOINT']}/openai/v1/",
)




# Simple test call
resp = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT, 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Introduce your self in short sentence."},
    ],
)

print(resp.choices[0].message.content)


I am an AI language model designed to assist with a variety of questions and tasks by providing information and generating text based on your input.


# Check the language syntax 

In [4]:
def normalize_query(text: str) -> str:
    resp = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {
                "role": "system",
                "content": (
                    "You understand poorly written Norwegian and English. "
                    "Correct spelling mistakes, interpret the intended meaning, "
                    "and rewrite the sentence clearly without adding new information."
                ),
            },
            {"role": "user", "content": text},
        ],
        temperature=0,
    )
    return resp.choices[0].message.content.strip()


# LLM planner: convert the natural language query to a structured JSON plan


In [5]:
import json


with open("system_prompt.txt", "r") as f:
    SYSTEM_PROMPT = f.read()

previous_input = None

def plan_spatial_query(nl_query: str) -> dict:
    resp = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": nl_query},
        ],
        temperature=0.0,
        max_tokens=300,
    )
    raw = resp.choices[0].message.content.strip()
    if raw.startswith("```"):
        raw = raw.strip("`")
        if raw.lower().startswith("json"):
            raw = raw[4:].strip()
    return json.loads(raw)



In [6]:
print(SYSTEM_PROMPT)

You are a GIS planning agent for a Nordic municipality.

Your role is to translate natural-language user requests into a structured
GIS execution plan.

You act strictly as a PLANNER.
Execution and database access are handled by external systems.
You NEVER execute SQL and NEVER access the database.

------------------------------------------------------------
OUTPUT CONTRACT (STRICT)
------------------------------------------------------------
You MUST output a single valid JSON object.
No explanations, no markdown, no text outside JSON.

The JSON MUST contain ALL of the following fields:

{
  "operation": string,
  "layer": string,
  "target_layer": string | null,
  "buffer_meters": number | null,
  "limit": number | null,
  "where_clause": string
}

All fields must exist.
If a field is not applicable, use null (or "" for where_clause).

------------------------------------------------------------
AVAILABLE GIS DATA (CONTEXT)
-----------------------------------------------------------

# Test LLM planner 

In [7]:
query = "Find 100 residential buildings larger than 500 m² within 200 meters of a river"
plan = plan_spatial_query(query)
plan


{'operation': 'select_buffer',
 'layer': 'buildings',
 'target_layer': 'flomsoner',
 'buffer_meters': 200,
 'limit': 100,
 'where_clause': "ST_Area(geom) > 500 AND type = 'residential'"}

In [8]:
def test(plan):
    return plan_spatial_query(plan)


In [9]:
test("Find the 10 buildings closest to bicycle routes")

{'operation': 'select_nearest',
 'layer': 'buildings',
 'target_layer': 'sykkelrute_senterlinje',
 'buffer_meters': None,
 'limit': 10,
 'where_clause': ''}

In [10]:
test("Find residential buildings within 200 meters of a river")

{'operation': 'select_buffer',
 'layer': 'buildings',
 'target_layer': 'flomsoner',
 'buffer_meters': 200,
 'limit': None,
 'where_clause': "type = 'residential'"}

In [11]:
test("Show buildings larger than 500 square meters")

{'operation': 'select_by_attribute',
 'layer': 'buildings',
 'target_layer': None,
 'buffer_meters': None,
 'limit': None,
 'where_clause': 'ST_Area(geom) > 500'}

In [12]:
test("Show 50 buildings")

{'operation': 'select_by_attribute',
 'layer': 'buildings',
 'target_layer': None,
 'buffer_meters': None,
 'limit': 50,
 'where_clause': ''}

In [13]:
def extract_municipality(text: str) -> str:
    resp = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {
                "role": "system",
                "content": (
                    "You extract Norwegian municipality names from user input. "
                    "Return ONLY the municipality name. "
                    "If no municipality is mentioned, return an empty string. "
                    "Do not guess."
                ),
            },
            {"role": "user", "content": text},
        ],
        temperature=0.0,
    )
    return resp.choices[0].message.content.strip()


In [14]:
extract_municipality("Find 100 residential houses within 200 meters of a river in Grimstad")

'Grimstad'

In [15]:
import requests

def lookup_kommune(name: str):
    url = "https://api.kartverket.no/kommuneinfo/v1/sok"
    params = {"knavn": name}

    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()

    avgrensningsboks = data["kommuner"][0]["avgrensningsboks"]
    coords = avgrensningsboks["coordinates"][0]

    lons = [pt[0] for pt in coords]
    lats = [pt[1] for pt in coords]

    return min(lons), min(lats), max(lons), max(lats)


In [16]:
df  = lookup_kommune("grimstad")



df

(8.30867587239, 58.081372035885, 9.067265283977, 58.491770310858)

# Process user input

In [17]:
def city_bbox_where_clause(min_lon, min_lat, max_lon, max_lat):
    return f"""
a.geom && ST_Transform(
    ST_MakeEnvelope(
        {min_lon}, {min_lat},
        {max_lon}, {max_lat},
        4326
    ),
    ST_SRID(a.geom)
)
""".strip()


In [18]:
city_bbox_where_clause(8.30867587239,58.081372035885,9.067265283977,58.491770310858)

'a.geom && ST_Transform(\n    ST_MakeEnvelope(\n        8.30867587239, 58.081372035885,\n        9.067265283977, 58.491770310858,\n        4326\n    ),\n    ST_SRID(a.geom)\n)'

In [19]:
FIELD_INFO = {
    
    # ****************************************************
    "layer": (
        "The PRIMARY dataset you want results from.\n"
        "Must match one of the allowed database layers:\n"
        "- buildings\n"
        "- flomsoner\n"
        "- buildings_sample\n"
        "- arealbruk_skogbonitet_sample\n"
        "- flomsoner_sample\n"
        "- sykkelrute_senterlinje_sample\n"
        "- skiloype_senterlinje\n"
        "- annenrute_senterlinje\n"
        "- annenruteinfo_tabell\n"
        "- arealbruk_skogbonitet\n"
        "- fotrute_senterlinje\n"
        "- fotruteinfo_tabell\n"
        "- ruteinfopunkt_posisjon\n"
        "- skiloypeinfo_tabell\n"
        "- sykkelrute_senterlinje\n"
        "- sykkelruteinfo_tabell"
    ),

}

def process_user_input(user_input):
    clean_text = normalize_query(user_input)

    plan = plan_spatial_query(clean_text)
    if not isinstance(plan, dict):
        return "Invalid plan"

    # --- municipality → bbox filter (kode, ikke LLM) ---
    municipality = extract_municipality(clean_text)
    city_filter = None

    if municipality:
        min_lon, min_lat, max_lon, max_lat = lookup_kommune(municipality)
        city_filter = city_bbox_where_clause(
            min_lon, min_lat, max_lon, max_lat
        )

    # --- combine where clauses ---
    existing = plan.get("where_clause")

    if city_filter:
        if existing and existing.strip():
            plan["where_clause"] = f"({existing}) AND ({city_filter})"
        else:
            plan["where_clause"] = city_filter

    # default limit
    if plan.get("limit") is None:
        plan["limit"] = 100

    # layer validation only
    if not plan.get("layer"):
        return (
            "⚠ Unknown or missing layer.\n\n"
            + FIELD_INFO["layer"]
            + f"\n\nYour input:\n  {user_input}"
        )

    return plan




In [20]:
df = process_user_input("Find 100 residential houses within 200 meters of a river in Grimstad.")

In [21]:
df

{'operation': 'select_buffer',
 'layer': 'buildings',
 'target_layer': 'flomsoner',
 'buffer_meters': 200,
 'limit': 100,
 'where_clause': "(type = 'residential') AND (a.geom && ST_Transform(\n    ST_MakeEnvelope(\n        8.30867587239, 58.081372035885,\n        9.067265283977, 58.491770310858,\n        4326\n    ),\n    ST_SRID(a.geom)\n))"}

# Converts the user’s prompt into an SQL query string.

In [22]:
def sql_select_limit_only(plan):
    layer = plan["layer"]
    limit = plan["limit"]

    return f"""
    SELECT
        a.*,
        ST_AsText(ST_Transform(a.geom, 4326)) AS wkt_geom
    FROM public.{layer} a
    LIMIT {limit};
    """.strip()




def sql_select_by_attribute(plan):
    layer = plan["layer"]
    where = plan.get("where_clause") or "TRUE"
    limit = plan["limit"]

    return f"""
    SELECT
        a.*,
        ST_AsText(ST_Transform(a.geom, 4326)) AS wkt_geom
    FROM public.{layer} a
    WHERE {where}
    LIMIT {limit};
    """.strip()



def sql_select_buffer(plan):
    layer = plan["layer"]
    target = plan["target_layer"]
    buffer_m = plan["buffer_meters"]
    where = plan.get("where_clause") or "TRUE"
    limit = plan["limit"]

    if not target or buffer_m is None:
        raise ValueError("select_buffer requires target_layer and buffer_meters")

    return f"""
    SELECT
        a.*,
        ST_AsText(ST_Transform(a.geom, 4326)) AS wkt_geom
    FROM public.{layer} a
    JOIN public.{target} b
      ON ST_DWithin(a.geom, b.geom, {buffer_m})
    WHERE {where}
    LIMIT {limit};
    """.strip()



def sql_select_intersect(plan):
    layer = plan["layer"]
    target = plan["target_layer"]
    where = plan.get("where_clause") or "TRUE"
    limit = plan["limit"]

    if not target:
        raise ValueError("select_intersect requires target_layer")

    return f"""
    SELECT
        a.*,
        ST_AsText(ST_Transform(a.geom, 4326)) AS wkt_geom
    FROM public.{layer} a
    JOIN public.{target} b
      ON ST_Intersects(a.geom, b.geom)
    WHERE {where}
    LIMIT {limit};
    """.strip()


def sql_select_nearest(plan):
    layer = plan["layer"]
    target = plan["target_layer"]
    limit = plan["limit"]

    if not target:
        raise ValueError("select_nearest requires target_layer")

    return f"""
    SELECT
        a.*,
        ST_AsText(ST_Transform(a.geom, 4326)) AS wkt_geom
    FROM public.{layer} a
    ORDER BY (
        SELECT MIN(ST_Distance(a.geom, b.geom))
        FROM public.{target} b
    )
    LIMIT {limit};
    """.strip()




In [23]:
df = process_user_input("Find 100 residential houses within 200 meters of a river in Kristiansand")

In [24]:
df

{'operation': 'select_buffer',
 'layer': 'buildings',
 'target_layer': 'flomsoner',
 'buffer_meters': 200,
 'limit': 100,
 'where_clause': "(type = 'residential') AND (a.geom && ST_Transform(\n    ST_MakeEnvelope(\n        7.527298530316, 57.802040927569,\n        8.371666458626, 58.368952685938,\n        4326\n    ),\n    ST_SRID(a.geom)\n))"}

In [25]:
def plan_to_sql(plan: dict) -> str:
    op = plan["operation"]

    if op == "select_limit_only":
        return sql_select_limit_only(plan)

    if op == "select_by_attribute":
        return sql_select_by_attribute(plan)

    if op == "select_buffer":
        return sql_select_buffer(plan)

    if op == "select_intersect":
        return sql_select_intersect(plan)

    if op == "select_nearest":
        return sql_select_nearest(plan)

    raise ValueError(f"Unsupported operation: {op}")


# Connect to the PostGIS database and run the SQL query, returning a DataFrame


In [26]:
from collections.abc import AsyncIterator
from contextlib import asynccontextmanager
from dataclasses import dataclass
import os

import psycopg2
import pandas as pd

from mcp.server.fastmcp import Context, FastMCP
from mcp.server.session import ServerSession


# ---------- Database ----------
class Database:
    def __init__(self, conn_str: str):
        self.conn_str = conn_str
        self.conn = None

    def connect(self) -> "Database":
        self.conn = psycopg2.connect(self.conn_str)
        return self

    def disconnect(self) -> None:
        if self.conn:
            self.conn.close()

    def query(self, sql: str) -> pd.DataFrame:
        with self.conn.cursor() as cur:
            cur.execute(sql)
            rows = cur.fetchall()
            cols = [desc[0] for desc in cur.description]
        return pd.DataFrame(rows, columns=cols)


# ---------- Lifespan context ----------
@dataclass
class AppContext:
    db: Database



@asynccontextmanager
async def app_lifespan(server: FastMCP):
    db = Database(os.environ["PGCONN_STRING"]).connect()
    try:
        yield AppContext(db=db)
    finally:
        db.disconnect()


# ---------- MCP server ----------
mcp = FastMCP("My App", lifespan=app_lifespan)


# ---------- Tool ----------
@mcp.tool(
    name="run_postgis_query",
    description="Run a SQL query on PostGIS and return a table"
)
def run_postgis_query2(
    ctx: Context[ServerSession, AppContext],
    sql: str
) -> list[dict]:
    df = ctx.request_context.lifespan_context.db.query(sql)
    return df.to_dict(orient="records")


In [27]:
db = Database(os.environ["PGCONN_STRING"])
db.connect()


In [28]:
print(db.conn)


<connection object at 0x7240a4fef9c0; dsn: 'user=kartai_ro password=xxx dbname=007workshop host=kartai-postgis-dev.postgres.database.azure.com port=5432', closed: 0>


In [27]:
db.conn.rollback()


In [28]:
db.query("SELECT * FROM buildings LIMIT 5;")


,gid,osm_id,code,fclass,name,type,geom
0,205278,184350253,1500,building,None,house,0106000020E96400000100000001030000000100000005...
1,205279,184350254,1500,building,None,terrace,0106000020E96400000100000001030000000100000008...
2,205280,184350255,1500,building,None,house,0106000020E96400000100000001030000000100000005...
3,205281,184350257,1500,building,None,terrace,0106000020E96400000100000001030000000100000009...
4,205282,184350258,1500,building,None,garage,0106000020E96400000100000001030000000100000005...


In [18]:
query = "SELECT * FROM buildings;"
db.query(query)


,gid,osm_id,code,fclass,name,type,geom
0,3706840,1012533458,1500,building,None,cabin,0106000020E96400000100000001030000000100000005...
1,3706841,1012533459,1500,building,None,cabin,0106000020E96400000100000001030000000100000005...
2,3706842,1012533460,1500,building,None,cabin,0106000020E96400000100000001030000000100000005...
3,3706843,1012533461,1500,building,None,hotel,0106000020E9640000010000000103000000010000000B...
4,3706844,1012533462,1500,building,None,cabin,0106000020E96400000100000001030000000100000009...
...,...,...,...,...,...,...,...
4158474,3706835,1012533453,1500,building,None,cabin,0106000020E96400000100000001030000000100000007...
4158475,3706836,1012533454,1500,building,None,cabin,0106000020E9640000010000000103000000010000000B...
4158476,3706837,1012533455,1500,building,None,cabin,0106000020E96400000100000001030000000100000005...
4158477,3706838,1012533456,1500,building,None,cabin,0106000020E9640000010000000103000000010000000A...


In [137]:
query = "SELECT relname AS table_name, pg_size_pretty(pg_total_relation_size(relid)) AS total_size FROM pg_catalog.pg_statio_user_tables ORDER BY pg_total_relation_size(relid) DESC;"
db.query(query)


,table_name,total_size
0,arealbruk_skogbonitet,3160 MB
1,buildings,1251 MB
2,flomsoner,343 MB
3,fotrute_senterlinje,141 MB
4,flomsoner_sample,54 MB
5,fotruteinfo_tabell,27 MB
6,sykkelrute_senterlinje,18 MB
7,arealbruk_skogbonitet_sample,17 MB
8,sykkelrute_senterlinje_sample,17 MB
9,skiloype_senterlinje,13 MB


In [203]:
query = "SELECT relname AS table_name, pg_size_pretty(pg_total_relation_size(relid)) AS total_size FROM pg_catalog.pg_statio_user_tables ORDER BY pg_total_relation_size(relid) DESC;"
db.query(query)


,table_name,total_size
0,arealbruk_skogbonitet,3160 MB
1,buildings,1251 MB
2,flomsoner,343 MB
3,fotrute_senterlinje,141 MB
4,flomsoner_sample,54 MB
5,fotruteinfo_tabell,27 MB
6,sykkelrute_senterlinje,18 MB
7,arealbruk_skogbonitet_sample,17 MB
8,sykkelrute_senterlinje_sample,17 MB
9,skiloype_senterlinje,13 MB


In [207]:
db.conn.rollback()


In [218]:
df = db.query("SELECT * FROM skiloypeinfo_tabell	;")
df


,gid,objtype,rutenavn,rutenummer,vedlikeholdsansvarlig,ruteinformasjon,spesialskiloypetype,gradering,rutetype,rutebetydning,tilpasning,skiloype_fk
0,1,Skiløype,Mosåsen lysløype,Ski_20160906_07,Vefsn kommune,None,,,NaN,NaN,,4cd7fc2d-d1ad-4946-9df2-831b9cabf8bb
1,2,Skiløype,Kjærstad Lysløype,Ski_20160906_05,Kjærstad IL,None,,,NaN,NaN,,e73bf270-0135-43c5-aa6c-54e785bf4275
2,3,Skiløype,Kjærstad Lysløype,Ski_20160906_05,Kjærstad IL,None,,,NaN,NaN,,ecfc7fa4-3683-4d4f-bd45-b2b713133678
3,4,Skiløype,Lysløype Sjåmoen,Ski_20160906_04,Vefsn kommune,None,,,NaN,NaN,,368778aa-1c55-477b-9383-f4e154474813
4,5,Skiløype,Ollmoen skiløype,Ski_20160906_01,Halsøy IL,None,,,NaN,NaN,,9be89712-000c-4a39-8a36-f030284b5f2f
...,...,...,...,...,...,...,...,...,...,...,...,...
12042,12043,Skiløype,Svarthopen lysløype,Ski_20250113_01,Brønnøysund idrettslag,None,,G,NaN,3.0,,93a5e61a-ece1-4686-a172-46f2e09891cd
12043,12044,Skiløype,Svarthopen lysløype,Ski_20250113_01,Brønnøysund idrettslag,None,,G,NaN,3.0,,93a5e61a-ece1-4686-a172-46f2e09891cd
12044,12045,Skiløype,Svarthopen lysløype,Ski_20250113_01,Brønnøysund idrettslag,None,,G,NaN,3.0,,31e094a2-fffb-48f3-b3eb-88633fa37781
12045,12046,Skiløype,Svarthopen lysløype,Ski_20250113_01,Brønnøysund idrettslag,None,,G,NaN,3.0,,31e094a2-fffb-48f3-b3eb-88633fa37781


In [19]:
db.query("SELECT * FROM arealbruk_skogbonitet LIMIT 10;")


,gid,artype,arskogbon,artreslag,arjordbr,arveget,areal,arkartstd,kilde,geom
0,1,50,98,39,98,99,1.188990e+04,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000016...
1,2,99,98,98,98,98,6.961873e+09,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000013...
2,3,82,98,98,98,98,2.572468e+09,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000260000006E...
3,4,50,98,39,98,55,1.165619e+05,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000007B...
4,5,50,98,39,98,51,2.457512e+04,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000002C...
5,6,50,98,39,98,55,2.094824e+04,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000028...
6,7,50,98,39,98,54,1.569405e+04,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000019...
7,8,50,98,39,98,51,2.748382e+04,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000031...
8,9,50,98,39,98,52,1.058840e+05,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000059...
9,890863,30,11,31,98,98,1.555363e+03,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000000C...


In [20]:
db.query("SELECT * FROM buildings LIMIT 10;")


,gid,osm_id,code,fclass,name,type,geom
0,3932990,1021346338,1500,building,None,house,0106000020E96400000100000001030000000100000009...
1,3932991,1021346339,1500,building,None,semidetached_house,0106000020E9640000010000000103000000010000000B...
2,3932992,1021346340,1500,building,None,semidetached_house,0106000020E9640000010000000103000000010000000B...
3,3932993,1021346341,1500,building,None,garage,0106000020E96400000100000001030000000100000005...
4,3932994,1021346342,1500,building,None,apartments,0106000020E96400000100000001030000000100000009...
5,3932995,1021346343,1500,building,None,house,0106000020E9640000010000000103000000010000000B...
6,3932996,1021346344,1500,building,None,garage,0106000020E96400000100000001030000000100000008...
7,3932997,1021346345,1500,building,None,house,0106000020E96400000100000001030000000100000011...
8,3932998,1021346346,1500,building,None,house,0106000020E9640000010000000103000000010000000D...
9,3932999,1021346347,1500,building,None,house,0106000020E9640000010000000103000000010000000D...


In [21]:
db.query("SELECT * FROM flomsoner LIMIT 10;")


,gid,objid,objtype,lavpunkt,gjentaksintervall,forstedigitaliseringsdato,noyaktighet,noyaktighethoyde,statusdato,flomsoneid,...,versjonid,datauttaksdato,opphav,symbolflom,malemetode,malemetodehoyde,statuskartlegging,shape_length,shape_area,geom
0,1,1,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,35.001523,87.041427,0106000020E9640000010000000103000000010000001D...
1,2,2,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,83.620072,177.187296,0106000020E96400000100000001030000000100000038...
2,3,3,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,50.070632,105.886299,0106000020E96400000100000001030000000100000032...
3,4,4,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,71.986928,249.904775,0106000020E96400000100000001030000000100000023...
4,5,5,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,150.904263,523.842873,0106000020E96400000100000001030000000100000078...
5,6,6,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,2206.783609,27629.307678,0106000020E96400000100000001030000000100000095...
6,7,7,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,42.220974,108.129053,0106000020E96400000100000001030000000100000013...
7,8,8,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,1784.441617,46832.829341,0106000020E96400000100000001030000000300000033...
8,9,9,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,14687.411504,936949.009108,0106000020E9640000010000000103000000100000006D...
9,10,10,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,5360.759072,430969.374874,0106000020E96400000100000001030000000800000080...


In [23]:
db.query("SELECT * FROM flomsoner_sample LIMIT 10;")


,gid,objid,objtype,lavpunkt,gjentaksintervall,forstedigitaliseringsdato,noyaktighet,noyaktighethoyde,statusdato,flomsoneid,...,versjonid,datauttaksdato,opphav,symbolflom,malemetode,malemetodehoyde,statuskartlegging,shape_length,shape_area,geom
0,43565,43555,FlomAreal,0,200,2006-09-01 00:00:00+00:00,36,None,2006-09-01 00:00:00+00:00,fs002_21,...,1.1,2025-03-16 09:10:05+00:00,NVE,1,61,None,1,126.529356,2.651606e+02,0106000020E96400000100000001030000000100000029...
1,60009,60068,FlomAreal,0,50,2007-10-18 00:00:00+00:00,36,None,2007-10-18 00:00:00+00:00,fs112_1,...,1.1,2025-03-16 09:13:54+00:00,NVE,1,61,None,1,8797.878073,2.005660e+05,0106000020E96400000100000001030000000A00000093...
2,11227,11234,FlomAreal,0,10,2003-12-05 00:00:00+00:00,36,None,2003-12-05 00:00:00+00:00,fs155_1,...,1.1,2025-03-16 09:03:26+00:00,NVE,1,61,None,1,144.731993,3.710203e+02,0106000020E96400000100000001030000000100000024...
3,42888,42934,FlomAreal,0,200,2006-06-06 00:00:00+00:00,36,None,2006-06-06 00:00:00+00:00,fs234_2,...,1.1,2025-03-16 09:10:01+00:00,NVE,1,61,None,1,5162.299394,1.675268e+05,0106000020E964000001000000010300000007000000B6...
4,28308,28156,FlomAreal,0,100,2002-12-10 00:00:00+00:00,36,None,2002-12-10 00:00:00+00:00,fs002_8,...,1.1,2025-03-16 09:06:31+00:00,NVE,1,61,None,1,14842.467629,1.538498e+06,0106000020E96400000100000001030000000E000000E8...
5,51072,50929,FlomAreal,1,200,2006-09-01 00:00:00+00:00,36,None,2006-09-01 00:00:00+00:00,fs002_21,...,1.1,2025-03-16 09:11:21+00:00,NVE,2,61,None,1,39.197685,8.495707e+01,0106000020E96400000100000001030000000100000015...
6,22722,22704,FlomAreal,1,10,2005-11-15 00:00:00+00:00,36,None,2005-11-15 00:00:00+00:00,fs121_1,...,1.1,2025-03-16 09:05:17+00:00,NVE,2,61,None,1,293.327963,4.883581e+02,0106000020E96400000100000001030000000100000058...
7,51200,51439,FlomAreal,1,200,2004-10-20 00:00:00+00:00,36,None,2004-10-20 00:00:00+00:00,fs002_2,...,1.1,2025-03-16 09:11:23+00:00,NVE,2,61,None,1,66.780494,1.504626e+02,0106000020E9640000010000000103000000010000001A...
8,9522,9340,FlomAreal,1,2100,2023-10-16 00:00:00+00:00,100,None,2024-05-03 00:00:00+00:00,fs121_3,...,1.1,2025-03-16 09:02:56+00:00,Norconsult AS,2,61,None,1,209.028339,7.829922e+02,0106000020E96400000100000001030000000100000022...
9,191,148,FlomAreal,0,500,2006-06-06 00:00:00+00:00,36,None,2006-06-06 00:00:00+00:00,fs234_2,...,1.1,2025-03-16 09:00:35+00:00,NVE,1,61,None,1,12.971835,9.508730e+00,0106000020E96400000100000001030000000100000017...


In [24]:
db.query("SELECT * FROM fotruteinfo_tabell LIMIT 10;")


,gid,objtype,rutenavn,rutenummer,vedlikeholdsansvarlig,ruteinformasjon,spesialfotrutetype,gradering,rutetype,rutebetydning,tilpasning,fotrute_fk
0,1,Fotrute,Heggelund-Sigfridstad-Karlstad,F_20221101_16,Romedal allmenning turstilag,None,,B,None,2,,c1ee09a6-d67c-4d5d-920e-8a4679c42a37
1,2,Fotrute,Heggelund-Sigfridstad-Karlstad,F_20221101_16,Romedal allmenning turstilag,None,,B,None,2,,670df9f6-8de1-4291-a865-3e31449745b7
2,3,Fotrute,Heggelund-Sigfridstad-Karlstad,F_20221101_16,Romedal allmenning turstilag,None,,B,None,2,,49803f0a-f422-4775-81fc-62a5ed0e62c8
3,4,Fotrute,Heggelund-Sigfridstad-Karlstad,F_20221101_16,Romedal allmenning turstilag,None,,B,None,2,,7fb9ec76-fedb-4400-bd29-631c6f09b87c
4,5,Fotrute,Skåltjennet-Barkilsætra,F_20221101_14,Romedal allmenning turstilag,None,,B,None,2,,84a14499-e676-48db-be1a-c96c333a805a
5,6,Fotrute,Turstier rundt Kvennstugua bygdetun,F_20221101_2,Kvennstugua bygdetun,None,,B,None,2,,754c5132-a93d-4fe8-a5a6-23d2d43288c0
6,7,Fotrute,Turstier rundt Kvennstugua bygdetun,F_20221101_2,Kvennstugua bygdetun,None,,B,None,2,,236c2db6-8838-4c06-88fc-856825e4ae23
7,8,Fotrute,Heggelund-Sigfridstad-Karlstad,F_20221101_16,Romedal allmenning turstilag,None,,B,None,2,,236c2db6-8838-4c06-88fc-856825e4ae23
8,9,Fotrute,Turstier rundt Kvennstugua bygdetun,F_20221101_2,Kvennstugua bygdetun,None,,B,None,2,,bec20206-c1af-4606-a10d-3f6bfa82a053
9,10,Fotrute,Heggelund-Sigfridstad-Karlstad,F_20221101_16,Romedal allmenning turstilag,None,,B,None,2,,bec20206-c1af-4606-a10d-3f6bfa82a053


In [22]:
db.query("SELECT * FROM fotrute_senterlinje LIMIT 10;")


,gid,objtype,skilting,anleggsnummer,uukoblingsid,belysning,lokalid,navnerom,versjonid,datafangstdato,...,informasjon,merking,rutefolger,underlagstype,rutebredde,trafikkbelastning,sesong,malemetode,shape_length,geom
0,1,Fotrute,JA,None,None,,c1ee09a6-d67c-4d5d-920e-8a4679c42a37,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2014-11-20 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,ST,None,None,None,,92,484.476705,0105000020E96400000100000001020000004300000070...
1,2,Fotrute,JA,None,None,,670df9f6-8de1-4291-a865-3e31449745b7,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2014-11-20 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,ST,None,None,None,,92,515.911710,0105000020E96400000100000001020000004B00000030...
2,3,Fotrute,JA,None,None,,49803f0a-f422-4775-81fc-62a5ed0e62c8,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2014-11-20 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,ST,None,None,None,,92,59.681554,0105000020E96400000100000001020000000600000090...
3,4,Fotrute,JA,None,None,,7fb9ec76-fedb-4400-bd29-631c6f09b87c,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2012-05-07 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,TR,None,None,None,,24,98.953422,0105000020E96400000100000001020000000E000000C0...
4,5,Fotrute,JA,None,None,,84a14499-e676-48db-be1a-c96c333a805a,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2010-06-16 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,TR,None,None,None,,24,627.273089,0105000020E96400000100000001020000004100000060...
5,6,Fotrute,JA,None,None,,754c5132-a93d-4fe8-a5a6-23d2d43288c0,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2014-11-20 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,ST,None,None,None,,92,388.746801,0105000020E96400000100000001020000001D00000050...
6,7,Fotrute,JA,None,None,,236c2db6-8838-4c06-88fc-856825e4ae23,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2021-06-01 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,ST,None,None,None,,24,159.089767,0105000020E96400000100000001020000003400000040...
7,8,Fotrute,JA,None,None,,bec20206-c1af-4606-a10d-3f6bfa82a053,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2021-06-01 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,ST,None,None,None,,24,145.407373,0105000020E96400000100000001020000002500000070...
8,9,Fotrute,JA,None,None,,89dd02b7-5cd0-4696-80b1-bcc8c9b27d57,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2021-06-01 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,ST,None,None,None,,24,8.129273,0105000020E964000001000000010200000005000000B0...
9,10,Fotrute,JA,None,None,,08b72c7d-4eb2-4752-b2a7-508a43d8f7eb,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-29 14:13:19.807707000,2023-01-16 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20230303",JA,ST,None,None,None,,92,1221.740829,0105000020E9640000010000000102000000C6000000D0...


In [25]:
db.query("SELECT * FROM sykkelrute_senterlinje LIMIT 10;")


,gid,objtype,skilting,anleggsnummer,uukoblingsid,belysning,lokalid,navnerom,versjonid,datafangstdato,...,informasjon,merking,rutefolger,underlagstype,rutebredde,trafikkbelastning,sesong,malemetode,shape_length,geom
0,1,Sykkelrute,,None,None,,02989546-4938-49c9-9405-906bafc7c54a,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1971-09-09 00:00:00+00:00,...,Generert fra Traktorvegkant,JA,TR,None,None,None,,60,523.414390,0105000020E9640000010000000102000000DF00000000...
1,2,Sykkelrute,,None,None,,090afebf-7920-4765-965e-c16024d70146,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1998-06-08 00:00:00+00:00,...,None,JA,BV,None,None,None,,22,3194.542971,0105000020E964000001000000010200000079000000D0...
2,3,Sykkelrute,,None,None,,0ea52391-5f2e-4bc1-aacd-2f7a6aeb0b38,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2009-11-18 00:00:00+00:00,...,Kombinert med sykkelrute,JA,GS,None,None,None,,60,32.429166,0105000020E96400000100000001020000000700000040...
3,4,Sykkelrute,,None,None,,386f947d-5bc1-45f0-8f5b-0a6124459861,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2014-09-17 00:00:00+00:00,...,None,JA,ST,None,None,None,,24,171.768400,0105000020E96400000100000001020000001F00000000...
4,5,Sykkelrute,,None,None,,253e48ab-d5e7-42b9-83f9-3a48c531e6cd,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2012-08-10 00:00:00+00:00,...,None,JA,TR,None,None,None,,24,35.215468,0105000020E96400000100000001020000000700000090...
5,6,Sykkelrute,,None,None,,3132db05-8e9a-4e53-a5fd-fb87df5d337e,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1996-06-13 00:00:00+00:00,...,None,JA,BV,None,None,None,,22,384.713777,0105000020E964000001000000010200000018000000E8...
6,7,Sykkelrute,,None,None,,12f58116-95e9-404b-ae98-4715e978914c,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1997-07-01 00:00:00+00:00,...,None,JA,ST,None,None,None,,22,3436.603532,0105000020E96400000100000001020000007C00000010...
7,8,Sykkelrute,,None,None,,157295ca-bd66-4fb3-b143-c7ec714b068b,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2016-08-09 00:00:00+00:00,...,None,JA,ST,None,None,None,,92,1263.105798,0105000020E964000001000000010200000076000000A8...
8,9,Sykkelrute,,None,None,,19bea04d-34f8-4f49-b4b7-67c5e6884357,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2008-08-19 00:00:00+00:00,...,None,JA,TR,None,None,None,,24,1255.101054,0105000020E96400000100000001020000008300000010...
9,10,Sykkelrute,,None,None,,97bd7a36-8f28-4f5e-9f4c-b39aeee3f41f,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2014-09-17 00:00:00+00:00,...,None,JA,ST,None,None,None,,24,146.498739,0105000020E96400000100000001020000001300000010...


In [26]:
db.query("SELECT * FROM arealbruk_skogbonitet_sample LIMIT 10;")


,gid,artype,arskogbon,artreslag,arjordbr,arveget,areal,arkartstd,kilde,geom
0,404887,30,18,31,98,98,17495.49076,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000037...
1,62029,30,11,32,98,98,88479.82210,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000072...
2,8595,50,98,39,98,55,60944.02784,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000005F...
3,766180,30,13,31,98,98,4149.25349,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000000E...
4,1332895,60,11,39,98,98,26014.77063,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000031...
5,1631254,30,11,33,98,98,2100.19337,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000009...
6,1504068,20,98,98,24,98,21021.49722,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000002F...
7,1819321,81,98,98,98,98,18311.25828,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000020...
8,1306302,30,11,33,98,98,72507.60282,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000056...
9,303328,50,98,39,98,55,53806.78316,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000003B...


In [27]:
db.query("SELECT * FROM sykkelrute_senterlinje_sample LIMIT 10;")


,gid,objtype,skilting,anleggsnummer,uukoblingsid,belysning,lokalid,navnerom,versjonid,datafangstdato,...,informasjon,merking,rutefolger,underlagstype,rutebredde,trafikkbelastning,sesong,malemetode,shape_length,geom
0,4612,Sykkelrute,JA,None,None,,42b20f86-7a29-4cd6-bd41-56f565dddf0f,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1998-05-11 00:00:00+00:00,...,Nasjonal sykkelrute hentet fra NVDB desember 2017,JA,BV,None,None,None,,22,958.990109,0105000020E964000001000000010200000060000000C0...
1,6487,Sykkelrute,,None,None,,59b8a83a-4055-443e-b1bc-e42ad0c93324,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2017-08-30 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20180410",JA,ST,None,None,None,,92,112.289226,0105000020E96400000100000001020000000B000000A0...
2,400,Sykkelrute,JA,None,None,,5e31ccb4-9e6a-48c8-9eac-d7b5ebca48a4,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1998-05-24 00:00:00+00:00,...,Nasjonal sykkelrute hentet fra NVDB desember 2017,JA,BV,None,None,None,,22,132.993062,0105000020E96400000100000001020000000700000030...
3,5300,Sykkelrute,JA,None,None,,ec0be669-3516-4bd0-b719-59ac9c97188b,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2011-04-18 00:00:00+00:00,...,Nasjonal sykkelrute hentet fra NVDB desember 2017,JA,SV,None,None,None,,24,144.356223,0105000020E96400000100000001020000001300000000...
4,2315,Sykkelrute,JA,None,None,,f92d19af-fb78-4a31-8591-6ef08721a6c4,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2017-12-14 00:00:00+00:00,...,None,JA,,None,None,None,,92,578.070913,0105000020E96400000100000001020000004400000040...
5,97,Sykkelrute,,None,None,,fb071e40-a56a-4805-95fc-f5e05d0f456f,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2008-08-19 00:00:00+00:00,...,None,JA,TR,None,None,None,,24,4016.585580,0105000020E96400000100000001020000002C010000C0...
6,5819,Sykkelrute,,None,None,,1ba3e896-2b3d-45be-8b51-01d89c46ec7f,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1989-07-01 00:00:00+00:00,...,"Geometri hentet fra Vbase, 20180508",JA,BV,None,None,None,,20,140.471644,0105000020E96400000100000001020000000E00000040...
7,3165,Sykkelrute,JA,None,None,,d6064130-5445-49fd-9abc-3b188a71f94f,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2014-07-22 00:00:00+00:00,...,"Geometri hentet fra Vbase, 20190821",JA,SB,None,None,None,,24,5664.776034,0105000020E9640000010000000102000000FA01000030...
8,10187,Sykkelrute,JA,None,None,,bcdeaad6-74f4-4b3f-9a24-8bf3e8194393,http://data.geonorge.no/TurruterNGIS/Turruter/so,2025-01-02 11:00:54.222472000,2006-07-15 00:00:00+00:00,...,"Geometri hentet fra Elveg, 20250102",NEI,BV,None,None,None,,92,35.111737,0105000020E964000001000000010200000003000000A0...
9,8538,Sykkelrute,JA,None,None,,b97d961f-f1ef-4c1f-a781-b5760549e370,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2020-09-03 00:00:00+00:00,...,"Geometri hentet fra TraktorvegSti, 20210225",JA,TR,None,None,None,,92,690.983091,0105000020E964000001000000010200000016000000E0...


In [28]:
db.query("SELECT * FROM skiloype_senterlinje LIMIT 10;")


,gid,objtype,skilting,anleggsnummer,uukoblingsid,belysning,lokalid,navnerom,versjonid,datafangstdato,...,rutebredde,trafikkbelastning,sesong,malemetode,antallskispor,skoytetrase,preparering,ryddebredde,shape_length,geom
0,1,Skiløype,,None,None,JA,4cd7fc2d-d1ad-4946-9df2-831b9cabf8bb,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,2013-07-24 00:00:00+00:00,...,None,None,,24,None,,PM,None,74.421103,0105000020E964000001000000010200000017000000B0...
1,2,Skiløype,,None,None,JA,e73bf270-0135-43c5-aa6c-54e785bf4275,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,1999-08-16 00:00:00+00:00,...,None,None,,24,None,,PM,None,91.913523,0105000020E96400000100000001020000001400000050...
2,3,Skiløype,,None,None,JA,ecfc7fa4-3683-4d4f-bd45-b2b713133678,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,2009-07-04 00:00:00+00:00,...,None,None,,24,None,,PM,None,78.796864,0105000020E964000001000000010200000008000000E0...
3,4,Skiløype,,None,None,JA,368778aa-1c55-477b-9383-f4e154474813,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,2009-07-04 00:00:00+00:00,...,None,None,,24,None,,PM,None,57.767037,0105000020E96400000100000001020000000600000040...
4,5,Skiløype,,None,None,,9be89712-000c-4a39-8a36-f030284b5f2f,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,2009-07-04 00:00:00+00:00,...,None,None,,24,None,,PM,None,147.011808,0105000020E96400000100000001020000000E000000B0...
5,6,Skiløype,,None,None,JA,801efc61-9805-4cfc-98fc-706e25783715,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,2014-05-27 00:00:00+00:00,...,None,None,,24,None,,PS,None,58.077748,0105000020E96400000100000001020000000B00000030...
6,7,Skiløype,,None,None,,2210800c-384f-4422-87fa-342895d07b48,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,1996-06-12 00:00:00+00:00,...,None,None,,22,None,,PM,None,382.413163,0105000020E96400000100000001020000002D00000098...
7,8,Skiløype,,None,None,JA,ac00f84f-fb53-4ad4-bee9-4ff1e202a04c,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,2009-07-04 00:00:00+00:00,...,None,None,,24,None,,PM,None,28.567904,0105000020E96400000100000001020000000300000050...
8,9,Skiløype,,None,None,JA,b7ceb00f-0eb2-4a01-9c88-5491f1cf616a,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,2014-07-21 00:00:00+00:00,...,None,None,,24,None,,PM,None,14.711254,0105000020E96400000100000001020000000200000030...
9,10,Skiløype,,None,None,JA,b1d0d26a-9682-4dd2-be3b-fa586704178b,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:26:28.556328000,1994-07-01 00:00:00+00:00,...,None,None,,20,None,,PM,None,86.240974,0105000020E96400000100000001020000000400000010...


In [30]:
db.query("SELECT * FROM buildings_sample LIMIT 10;")


,gid,osm_id,code,fclass,name,type,geom
0,302517,256981899,1500,building,None,house,0106000020E96400000100000001030000000100000011...
1,1113866,917527996,1500,building,None,house,0106000020E96400000100000001030000000100000009...
2,2741098,955796078,1500,building,None,house,0106000020E96400000100000001030000000100000009...
3,484043,474890912,1500,building,None,cabin,0106000020E96400000100000001030000000100000007...
4,921733,758449296,1500,building,None,farm_auxiliary,0106000020E96400000100000001030000000100000005...
5,2030431,939653608,1500,building,None,shed,0106000020E96400000100000001030000000100000005...
6,3394727,1001002724,1500,building,None,house,0106000020E96400000100000001030000000100000009...
7,1511943,925560420,1500,building,None,barn,0106000020E96400000100000001030000000100000005...
8,250466,219432263,1500,building,None,house,0106000020E9640000010000000103000000010000000C...
9,1131945,917909843,1500,building,None,garage,0106000020E96400000100000001030000000100000005...


In [29]:
db.query("SELECT * FROM ruteinfopunkt_posisjon LIMIT 10;")


,gid,objtype,ruteinfoid,vedlikeholdsansvarlig,anleggsnummer,uukoblingsid,lokalid,navnerom,versjonid,datafangstdato,...,noyaktighet,opphav,omradeid,originaldatavert,kopidato,informasjon,tilrettelegging,sesong,malemetode,geom
0,1,RuteInfoPunkt,None,Båtsfjord kommune,None,None,010e1521-2246-49b3-a151-7525ae9c0f0b,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,2010-08-10 00:00:00+00:00,...,500,Rett i kartet,9999,Kartverket,2025-03-02 04:30:43+00:00,"Start for turløypene til Skrovnes lykt, til Li...",22,,45,0104000020E9640000010000000101000000907EFB5A4C...
1,2,RuteInfoPunkt,5255,Andre,None,None,01acb05b-c2a3-48f7-a632-8346703a8ef6,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,2010-02-19 00:00:00+00:00,...,1500,Lerkevannskoia,9999,Kartverket,2025-03-02 04:30:44+00:00,Rastebu,12,,82,0104000020E9640000010000000101000000343333D399...
2,3,RuteInfoPunkt,5268,Andre,None,None,0267a3d2-ec2e-41ae-b529-2dc37687c9ca,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,1999-07-01 00:00:00+00:00,...,1500,Gjøkvasskoia,9999,Kartverket,2025-03-02 04:30:44+00:00,Rastebu,12,,50,0104000020E9640000010000000101000000D0915C2E90...
3,4,RuteInfoPunkt,5266,Andre,None,None,0910ef71-1afb-4c8f-8b4c-59b785561980,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,1999-07-01 00:00:00+00:00,...,1500,Ellentjørnkoia,9999,Kartverket,2025-03-02 04:30:44+00:00,Rastebu,12,,50,0104000020E964000001000000010100000040F163DC62...
4,5,RuteInfoPunkt,5113,Andre,None,None,0f4f4fc6-3c84-4799-83d5-4c42542315fe,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,2010-02-19 00:00:00+00:00,...,1500,Føllvannskoia,9999,Kartverket,2025-03-02 04:30:44+00:00,Rastebu,12,,82,0104000020E964000001000000010100000070A3019CEE...
5,6,RuteInfoPunkt,5185,Andre,None,None,104f8886-6f4c-4198-b5a1-5be713089bf7,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,2000-12-19 00:00:00+00:00,...,2500,Cunovuohppi,9999,Kartverket,2025-03-02 04:30:44+00:00,None,42,,82,0104000020E964000001000000010100000088C954C126...
6,7,RuteInfoPunkt,None,Sykkelbyen Alta,None,None,12f904ff-f5c6-4d5f-b440-f564ded05887,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,2016-08-09 00:00:00+00:00,...,500,Rett i kartet,9999,Kartverket,2025-03-02 04:30:44+00:00,Biggas parkering - Sykkelrute Stifjell,22,,92,0104000020E9640000010000000101000000E8839EEDB7...
7,8,RuteInfoPunkt,5231,Andre,None,None,138b0275-2fe5-47b9-8b4d-d0ffa931f129,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,1973-01-01 00:00:00+00:00,...,2500,Ravnastua,9999,Kartverket,2025-03-02 04:30:44+00:00,None,42,,82,0104000020E9640000010000000101000000A8F1D22D56...
8,9,RuteInfoPunkt,1002,DNT,None,None,1eb7895f-8f52-4a09-a688-d31c40330301,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,2002-07-01 00:00:00+00:00,...,2500,Reinbukkelvhytta,9999,Kartverket,2025-03-02 04:30:44+00:00,Rastebu,12,,82,0104000020E9640000010000000101000000D8F97E2A69...
9,10,RuteInfoPunkt,5234,Andre,None,None,21367a9c-5a9d-4893-b1eb-688796ac35e5,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:28:50.210778000,1997-08-01 00:00:00+00:00,...,1500,Rundvannshytta,9999,Kartverket,2025-03-02 04:30:44+00:00,None,44,,23,0104000020E9640000010000000101000000D856ECAFAB...


In [31]:
db.query("SELECT * FROM sykkelruteinfo_tabell LIMIT 10;")


,gid,objtype,rutenavn,rutenummer,vedlikeholdsansvarlig,ruteinformasjon,spesialsykkelrutetype,gradering,rutetype,rutebetydning,tilpasning,sykkelrute_fk
0,1,Sykkelrute,Lievlan,Sy_20151112_01,Sirma IL,None,None,,None,None,,02989546-4938-49c9-9405-906bafc7c54a
1,2,Sykkelrute,Govdagáldu,Sy_20151112_02,Sirma IL,None,None,,None,None,,02989546-4938-49c9-9405-906bafc7c54a
2,3,Sykkelrute,Jotkarunden,12,Sykkelbyen Alta,None,None,,None,None,,090afebf-7920-4765-965e-c16024d70146
3,4,Sykkelrute,Karasjok Váljohka,Sy_20151108_01,Karasjok sykkelklubb,None,None,,None,None,,0ea52391-5f2e-4bc1-aacd-2f7a6aeb0b38
4,5,Sykkelrute,Suollovárri/Holmfjel,Sy_20151112_04,Sirma IL,None,None,,None,None,,386f947d-5bc1-45f0-8f5b-0a6124459861
5,6,Sykkelrute,Folkestien/ Álbmotbá,Sy_20160711_01,Saarela vel,None,None,,None,None,,253e48ab-d5e7-42b9-83f9-3a48c531e6cd
6,7,Sykkelrute,Folkestien/ Álbmotbá,Sy_20160711_01,Saarela vel,None,None,,None,None,,3132db05-8e9a-4e53-a5fd-fb87df5d337e
7,8,Sykkelrute,Hánájávri,Sy_20151112_03,Sirma IL,None,None,,None,None,,12f58116-95e9-404b-ae98-4715e978914c
8,9,Sykkelrute,Jotkarunden,12,Sykkelbyen Alta,None,None,,None,None,,157295ca-bd66-4fb3-b143-c7ec714b068b
9,10,Sykkelrute,Hánájávri,Sy_20151112_03,Sirma IL,None,None,,None,None,,19bea04d-34f8-4f49-b4b7-67c5e6884357


In [32]:
db.query("SELECT * FROM annenrute_senterlinje LIMIT 10;")


,gid,objtype,skilting,anleggsnummer,uukoblingsid,belysning,lokalid,navnerom,versjonid,datafangstdato,...,informasjon,merking,rutefolger,underlagstype,rutebredde,trafikkbelastning,sesong,malemetode,shape_length,geom
0,1,AnnenRute,JA,None,None,,8244e1ef-d249-4964-9479-6a37eae7e03e,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,3096.120276,0105000020E964000001000000010200000018000000AC...
1,2,AnnenRute,JA,None,None,,2c3d344b-9af5-47c8-b117-9429944a05b4,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,13889.917010,0105000020E96400000100000001020000003C00000058...
2,3,AnnenRute,JA,None,None,,3c5a7a6a-50df-411d-bf86-621273ff2715,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,3991.721159,0105000020E964000001000000010200000018000000BC...
3,4,AnnenRute,JA,None,None,,bad42759-872f-4aeb-8397-ee0f3e74d5b4,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,7278.905505,0105000020E96400000100000001020000003700000094...
4,5,AnnenRute,JA,None,None,,f83872aa-f8ff-4828-b9fd-190ea4491a39,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,2569.491609,0105000020E964000001000000010200000017000000C4...
5,6,AnnenRute,JA,None,None,,35db4737-26b7-4fe2-bf0b-68edc966af44,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,4824.533697,0105000020E96400000100000001020000001C00000018...
6,7,AnnenRute,JA,None,None,,1f716ce5-6a9f-4bfa-a0ad-94cff06a3882,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,19457.105166,0105000020E96400000100000001020000003E00000018...
7,8,AnnenRute,JA,None,None,,1984743b-4866-4086-8d5d-749a31faf2a2,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,2786.720020,0105000020E96400000100000001020000001A0000005C...
8,9,AnnenRute,JA,None,None,,4019b99e-7e54-40a8-813d-7d84251b50cb,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,8189.706384,0105000020E9640000010000000102000000230000001C...
9,10,AnnenRute,JA,None,None,,eeda1321-2887-428e-8342-ac72d40b4c6f,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:24:28.300970000,2021-01-07 00:00:00+00:00,...,None,NEI,VV,None,None,None,,45,7067.157529,0105000020E96400000100000001020000001A00000064...


In [33]:
db.query("SELECT * FROM skiloypeinfo_tabell LIMIT 10;")


,gid,objtype,rutenavn,rutenummer,vedlikeholdsansvarlig,ruteinformasjon,spesialskiloypetype,gradering,rutetype,rutebetydning,tilpasning,skiloype_fk
0,1,Skiløype,Mosåsen lysløype,Ski_20160906_07,Vefsn kommune,None,,,None,NaN,,4cd7fc2d-d1ad-4946-9df2-831b9cabf8bb
1,2,Skiløype,Kjærstad Lysløype,Ski_20160906_05,Kjærstad IL,None,,,None,NaN,,e73bf270-0135-43c5-aa6c-54e785bf4275
2,3,Skiløype,Kjærstad Lysløype,Ski_20160906_05,Kjærstad IL,None,,,None,NaN,,ecfc7fa4-3683-4d4f-bd45-b2b713133678
3,4,Skiløype,Lysløype Sjåmoen,Ski_20160906_04,Vefsn kommune,None,,,None,NaN,,368778aa-1c55-477b-9383-f4e154474813
4,5,Skiløype,Ollmoen skiløype,Ski_20160906_01,Halsøy IL,None,,,None,NaN,,9be89712-000c-4a39-8a36-f030284b5f2f
5,6,Skiløype,Lysløypa,Ski_20160628,IL Splint,Turskiltprosjektet,,,None,NaN,,801efc61-9805-4cfc-98fc-706e25783715
6,7,Skiløype,Ulvsvågskaret,Ski_26,Sørulf IL/Hamarøy kommune,None,RL,B,None,2.0,,2210800c-384f-4422-87fa-342895d07b48
7,8,Skiløype,Mosåsen lysløype,Ski_20160906_07,Vefsn kommune,None,,,None,NaN,,ac00f84f-fb53-4ad4-bee9-4ff1e202a04c
8,9,Skiløype,Kjærstad Lysløype,Ski_20160906_05,Kjærstad IL,None,,,None,NaN,,b7ceb00f-0eb2-4a01-9c88-5491f1cf616a
9,10,Skiløype,Kjærstad Lysløype,Ski_20160906_05,Kjærstad IL,None,,,None,NaN,,b1d0d26a-9682-4dd2-be3b-fa586704178b


In [34]:
db.query("SELECT * FROM annenruteinfo_tabell LIMIT 10;")


,gid,objtype,rutenavn,rutenummer,vedlikeholdsansvarlig,ruteinformasjon,spesialannenrutetype,gradering,rutetype,rutebetydning,tilpasning,annenrute_fk
0,1,AnnenRute,Lille Ropelvvatnet,AN_20210107_3,Sør-Varanger turlag,Padleperle 3 er med i brosjyre om kajakkturer ...,1,G,None,2,,8244e1ef-d249-4964-9479-6a37eae7e03e
1,2,AnnenRute,"Sølferbotn, Skogerøya",AN_20210107_12,Sør-Varanger turlag,Padleperle 12 er med i brosjyre om kajakkturer...,1,R,None,2,,2c3d344b-9af5-47c8-b117-9429944a05b4
2,3,AnnenRute,"Brannholmen, Munkefjorden",AN_20210107_7,Sør-Varanger turlag,Padleperle 7 er med i brosjyre om kajakkturer ...,1,B,None,2,,3c5a7a6a-50df-411d-bf86-621273ff2715
3,4,AnnenRute,"Sandbukt, Bugøyfjord",AN_20210107_9,Sør-Varanger turlag,Padleperle 9 er med i brosjyre om kajakkturer ...,1,B,None,2,,bad42759-872f-4aeb-8397-ee0f3e74d5b4
4,5,AnnenRute,"Kvalneset, Pasvikelva",AN_20210107_2,Sør-Varanger turlag,Padleperle 2 er med i brosjyre om kajakkturer ...,1,G,None,2,,f83872aa-f8ff-4828-b9fd-190ea4491a39
5,6,AnnenRute,"Storsteinen, Bøkfjorden",AN_20210107_6,Sør-Varanger turlag,Padleperle 6 er med i brosjyre om kajakkturer ...,1,B,None,2,,35db4737-26b7-4fe2-bf0b-68edc966af44
6,7,AnnenRute,"Brashamna, Kjøfjorden",AN_20210107_8,Sør-Varanger turlag,Padleperle 8 er med i brosjyre om kajakkturer ...,1,R,None,2,,1f716ce5-6a9f-4bfa-a0ad-94cff06a3882
7,8,AnnenRute,Svartakselvannet,AN_20210107_4,Sør-Varanger turlag,Padleperle 4 er med i brosjyre om kajakkturer ...,1,G,None,2,,1984743b-4866-4086-8d5d-749a31faf2a2
8,9,AnnenRute,Langfjordvatnet,AN_20210107_5,Sør-Varanger turlag,Padleperle 5 er med i brosjyre om kajakkturer ...,1,G,None,2,,1984743b-4866-4086-8d5d-749a31faf2a2
9,10,AnnenRute,Lille Jarfjord i Jarfjorden,AN_20210107_11,Sør-Varanger turlag,Padleperle 11 er med i brosjyre om kajakkturer...,1,R,None,2,,4019b99e-7e54-40a8-813d-7d84251b50cb


In [217]:
df = db.query("SELECT * FROM arealbruk_skogbonitet_sample;")
df


,gid,artype,arskogbon,artreslag,arjordbr,arveget,areal,arkartstd,kilde,geom
0,404887,30,18,31,98,98,17495.49076,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000037...
1,62029,30,11,32,98,98,88479.82210,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000072...
2,8595,50,98,39,98,55,60944.02784,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000005F...
3,766180,30,13,31,98,98,4149.25349,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000000E...
4,1332895,60,11,39,98,98,26014.77063,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000031...
...,...,...,...,...,...,...,...,...,...,...
9995,996407,50,98,39,98,54,40128.58285,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E96400000100000001030000000100000054...
9996,1820854,50,98,39,98,55,146322.57351,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E964000001000000010300000001000000A2...
9997,1076616,30,11,31,98,98,50689.11696,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000004D...
9998,1147792,60,99,39,98,98,404092.08791,AR50,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,0106000020E9640000010000000103000000010000003F...


In [216]:
df = db.query("SELECT * FROM annenruteinfo_tabell;")
df


,gid,objtype,rutenavn,rutenummer,vedlikeholdsansvarlig,ruteinformasjon,spesialannenrutetype,gradering,rutetype,rutebetydning,tilpasning,annenrute_fk
0,1,AnnenRute,Lille Ropelvvatnet,AN_20210107_3,Sør-Varanger turlag,Padleperle 3 er med i brosjyre om kajakkturer ...,1.0,G,NaN,2.0,,8244e1ef-d249-4964-9479-6a37eae7e03e
1,2,AnnenRute,"Sølferbotn, Skogerøya",AN_20210107_12,Sør-Varanger turlag,Padleperle 12 er med i brosjyre om kajakkturer...,1.0,R,NaN,2.0,,2c3d344b-9af5-47c8-b117-9429944a05b4
2,3,AnnenRute,"Brannholmen, Munkefjorden",AN_20210107_7,Sør-Varanger turlag,Padleperle 7 er med i brosjyre om kajakkturer ...,1.0,B,NaN,2.0,,3c5a7a6a-50df-411d-bf86-621273ff2715
3,4,AnnenRute,"Sandbukt, Bugøyfjord",AN_20210107_9,Sør-Varanger turlag,Padleperle 9 er med i brosjyre om kajakkturer ...,1.0,B,NaN,2.0,,bad42759-872f-4aeb-8397-ee0f3e74d5b4
4,5,AnnenRute,"Kvalneset, Pasvikelva",AN_20210107_2,Sør-Varanger turlag,Padleperle 2 er med i brosjyre om kajakkturer ...,1.0,G,NaN,2.0,,f83872aa-f8ff-4828-b9fd-190ea4491a39
...,...,...,...,...,...,...,...,...,...,...,...,...
2563,2564,AnnenRute,Hagasand Fjordrunden,Rute 29,DNT | DNT Drammen og Omegn,None,1.0,B,NaN,1.0,,a6ddd2f3-d7a5-401f-bc69-20ba9490cce8
2564,2565,AnnenRute,Vågsmyra båthavn - Lagunen i Skjervika,1506039,Molde kommune,Padlerute,1.0,,1.0,NaN,,4205dc21-5ec9-4bd7-9763-7c9479726a36
2565,2566,AnnenRute,Industriruta til Midsund sentrum,1506038,Molde kommune,Padlerute,1.0,,1.0,NaN,,27629bdc-5aef-44b2-8318-47bc79e118fc
2566,2567,AnnenRute,Ulnestangen - Dynholmen - Eidsand,AR_270624_01,Brandbu IF kajakk,None,NaN,,NaN,NaN,,ac0f0bae-8b74-41e3-a493-7721aa14aec0


# Execute the GIS plan in PostGIS and return the result as a DataFrame


In [29]:
from typing import Optional
import pandas as pd

def run_gis_query(query: str, db: Database) -> Optional[pd.DataFrame]:
    plan = process_user_input(query)

    if isinstance(plan, str):
        print(plan)
        return None

    sql = plan_to_sql(plan)
    #print("SQL:", sql)

    return db.query(sql)


In [30]:


from shapely import wkt
import folium
from shapely.geometry import Polygon, MultiPolygon


def normalize_geom(g):
    if isinstance(g, Polygon):
        return [g]
    if isinstance(g, MultiPolygon):
        return list(g.geoms)
    return []

def showMap(df):
    first_geom = wkt.loads(df["wkt_geom"].iloc[0])
    m = folium.Map(location=[first_geom.centroid.y, first_geom.centroid.x], zoom_start=16)    
    
    for w in df["wkt_geom"]:
        geom = wkt.loads(w)
        for poly in normalize_geom(geom):
            folium.GeoJson(
                poly.__geo_interface__,
                style_function=lambda x: {
                    "color": "red",
                    "weight": 2,
                    "fillColor": "yellow",
                    "fillOpacity": 0.3,
                },
            ).add_to(m)
    
    return m

In [43]:
run_gis_query("Find 100 residential houses within 200 meters of a river in Kristiansand.", db)

[12/16/25 01:52:13] INFO     HTTP Request: POST                                                     ]8;id=808265;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=399794;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=994563;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=642920;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=128252;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=930413;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

,gid,osm_id,code,fclass,name,type,geom,wkt_geom
0,1363875,921036209,1500,building,None,residential,0106000020E9640000010000000103000000010000000F...,MULTIPOLYGON(((7.971774700000005 58.2606905000...
1,1363875,921036209,1500,building,None,residential,0106000020E9640000010000000103000000010000000F...,MULTIPOLYGON(((7.971774700000005 58.2606905000...
2,1363875,921036209,1500,building,None,residential,0106000020E9640000010000000103000000010000000F...,MULTIPOLYGON(((7.971774700000005 58.2606905000...
3,1363875,921036209,1500,building,None,residential,0106000020E9640000010000000103000000010000000F...,MULTIPOLYGON(((7.971774700000005 58.2606905000...
4,1363875,921036209,1500,building,None,residential,0106000020E9640000010000000103000000010000000F...,MULTIPOLYGON(((7.971774700000005 58.2606905000...
5,1363875,921036209,1500,building,None,residential,0106000020E9640000010000000103000000010000000F...,MULTIPOLYGON(((7.971774700000005 58.2606905000...
6,1363875,921036209,1500,building,None,residential,0106000020E9640000010000000103000000010000000F...,MULTIPOLYGON(((7.971774700000005 58.2606905000...


In [40]:
run_gis_query("Find residential buildings within 200 meters of a rive", db)

[12/15/25 19:20:48] INFO     HTTP Request: POST                                                     ]8;id=524766;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=813958;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

[12/15/25 19:20:49] INFO     HTTP Request: POST                                                     ]8;id=100097;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=373717;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

plan: {'operation': 'select_buffer', 'layer': 'buildings', 'target_layer': 'flomsoner', 'buffer_meters': 200, 'limit': 100, 'where_clause': ''}
SQL: SELECT
        a.*,
        ST_AsText(ST_Transform(a.geom, 4326)) AS wkt_geom
    FROM public.buildings a
    JOIN public.flomsoner b
      ON ST_DWithin(a.geom, b.geom, 200)
    WHERE TRUE
    LIMIT 100;


,gid,osm_id,code,fclass,name,type,geom,wkt_geom
0,155448,165253015,1500,building,None,office,0106000020E9640000010000000103000000010000001B...,"MULTIPOLYGON(((10.391814 63.432916,10.39181600..."
1,155448,165253015,1500,building,None,office,0106000020E9640000010000000103000000010000001B...,"MULTIPOLYGON(((10.391814 63.432916,10.39181600..."
2,155448,165253015,1500,building,None,office,0106000020E9640000010000000103000000010000001B...,"MULTIPOLYGON(((10.391814 63.432916,10.39181600..."
3,155448,165253015,1500,building,None,office,0106000020E9640000010000000103000000010000001B...,"MULTIPOLYGON(((10.391814 63.432916,10.39181600..."
4,155448,165253015,1500,building,None,office,0106000020E9640000010000000103000000010000001B...,"MULTIPOLYGON(((10.391814 63.432916,10.39181600..."
...,...,...,...,...,...,...,...,...
95,158339,167115974,1500,building,None,school,0106000020E9640000010000000103000000010000001B...,MULTIPOLYGON(((10.929737700000002 59.221761199...
96,158339,167115974,1500,building,None,school,0106000020E9640000010000000103000000010000001B...,MULTIPOLYGON(((10.929737700000002 59.221761199...
97,158339,167115974,1500,building,None,school,0106000020E9640000010000000103000000010000001B...,MULTIPOLYGON(((10.929737700000002 59.221761199...
98,158339,167115974,1500,building,None,school,0106000020E9640000010000000103000000010000001B...,MULTIPOLYGON(((10.929737700000002 59.221761199...


In [249]:
run_gis_query("Find bulidings inside flood zones", db)

[12/15/25 13:23:46] INFO     HTTP Request: POST                                                     ]8;id=936654;file:///usr/local/lib/python3.10/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=780464;file:///usr/local/lib/python3.10/dist-packages/httpx/_client.py#1027\1027]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

[12/15/25 13:23:47] INFO     HTTP Request: POST                                                     ]8;id=590679;file:///usr/local/lib/python3.10/dist-packages/httpx/_client.py\_client.py]8;;\:]8;id=191933;file:///usr/local/lib/python3.10/dist-packages/httpx/_client.py#1027\1027]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

,gid,osm_id,code,fclass,name,type,geom,wkt_geom
0,2028,36938879,1500,building,Tynset Flyplass,transportation,0106000020E96400000100000001030000000100000009...,"MULTIPOLYGON(((10.6752223 62.25882310000001,10..."
1,2028,36938879,1500,building,Tynset Flyplass,transportation,0106000020E96400000100000001030000000100000009...,"MULTIPOLYGON(((10.6752223 62.25882310000001,10..."
2,2028,36938879,1500,building,Tynset Flyplass,transportation,0106000020E96400000100000001030000000100000009...,"MULTIPOLYGON(((10.6752223 62.25882310000001,10..."
3,2028,36938879,1500,building,Tynset Flyplass,transportation,0106000020E96400000100000001030000000100000009...,"MULTIPOLYGON(((10.6752223 62.25882310000001,10..."
4,2029,36938881,1500,building,None,hangar,0106000020E96400000100000001030000000100000005...,MULTIPOLYGON(((10.674991500000003 62.258489499...
...,...,...,...,...,...,...,...,...
95,3103,42310672,1500,building,None,retail,0106000020E9640000010000000103000000010000001D...,MULTIPOLYGON(((10.520700999999999 59.892315000...
96,3103,42310672,1500,building,None,retail,0106000020E9640000010000000103000000010000001D...,MULTIPOLYGON(((10.520700999999999 59.892315000...
97,4681,51750662,1500,building,None,retail,0106000020E9640000010000000103000000010000000D...,"MULTIPOLYGON(((10.0278238 59.7478838,10.027897..."
98,4681,51750662,1500,building,None,retail,0106000020E9640000010000000103000000010000000D...,"MULTIPOLYGON(((10.0278238 59.7478838,10.027897..."


In [ ]:
# run_gis_query("Find the 10 buildings closest to bicycle routes", db)

[12/15/25 14:26:49] INFO     HTTP Request: POST                                                     ]8;id=894520;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=452584;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=870034;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=625777;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

plan: {'operation': 'select_nearest', 'layer': 'buildings', 'target_layer': 'sykkelrute_senterlinje', 'buffer_meters': 500, 'limit': 10, 'where_clause': ''}
SQL: SELECT
        a.*,
        ST_AsText(ST_Transform(a.geom, 4326)) AS wkt_geom
    FROM public.buildings a
    JOIN LATERAL (
        SELECT b.geom
        FROM public.sykkelrute_senterlinje b
        ORDER BY a.geom <-> b.geom
        LIMIT 1
    ) b_nearest ON true
    ORDER BY a.geom <-> b_nearest.geom
    LIMIT 10;


# Send the query to the GIS agent and display the first results


In [51]:
df = run_gis_query("Find 3000 buildings in Kristiansand som nær flomw soner.", db)

showMap(df)


[12/16/25 02:02:24] INFO     HTTP Request: POST                                                     ]8;id=33085;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=451281;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

[12/16/25 02:02:25] INFO     HTTP Request: POST                                                     ]8;id=451676;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=712391;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=872925;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=997483;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

In [52]:
df = run_gis_query("Find 100 residential houses within 200 meters of a river in Kristiansand.", db)

showMap(df)


[12/16/25 02:03:33] INFO     HTTP Request: POST                                                     ]8;id=228546;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=636502;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

[12/16/25 02:03:34] INFO     HTTP Request: POST                                                     ]8;id=876458;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=394844;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=549114;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=18991;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

In [1]:
query = "Find 500 residential houses within 200 meters of a river in Fredrikstad."
df = run_gis_query(query, db)

showMap(df)


NameError: name 'run_gis_query' is not defined

In [60]:




query = "Vis meg 500 hus som er over 500 m² i Grimstad"
df = run_gis_query(query, db)

showMap(df)


[12/16/25 02:18:00] INFO     HTTP Request: POST                                                     ]8;id=361467;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=861329;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

[12/16/25 02:18:01] INFO     HTTP Request: POST                                                     ]8;id=713237;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=439611;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=57626;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=957234;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

In [ ]:







query = "Find the 10 buildings closest to bicycle routes"
df = run_gis_query(query, db)

showMap(df)


[12/16/25 02:21:54] INFO     HTTP Request: POST                                                     ]8;id=161959;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=295880;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

[12/16/25 02:21:55] INFO     HTTP Request: POST                                                     ]8;id=273752;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=73909;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=167501;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=39679;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

# Simple CLI chat loop that sends user queries to the GIS agent and prints the results


In [ ]:
def chat_loop(db):
    print("GIS agent chat – type 'quit' to stop.\n")

    while True:
        user_q = input("You: ").strip()
        print("'''''''''''''''''''''''''''''''''''''''''''")

        if user_q.lower() in ("quit", "exit", "q"):
            print("Welcome back")
            break

        df = run_gis_query(user_q, db)

        if df is None:
            continue

        if df.empty:
            print("No results.")
            continue

        display(showMap(df))
chat_loop(db)

GIS agent chat – type 'quit' to stop.



You:  Find the 10 buildings closest to bicycle routes i Grimstad


'''''''''''''''''''''''''''''''''''''''''''


[12/16/25 02:32:48] INFO     HTTP Request: POST                                                     ]8;id=932517;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=949367;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

[12/16/25 02:32:49] INFO     HTTP Request: POST                                                     ]8;id=546368;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=698764;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST                                                     ]8;id=209707;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=668676;file:///home/matinm/.local/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\
                             https://kartai-openai.openai.azure.com/openai/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     